In [1]:
import cv2
import numpy as np
import os

# 图片处理

## 边缘检测

①高斯模糊 - GaussianBlur
②灰度转换 - cvtColor
③计算梯度 – Sobel/Scharr
④非最大信号抑制
⑤高低阈值输出二值图像

In [ ]:
rawImage = cv2.imread(r'E:\Machine Learning\cannyfile\DSCN1394.JPG')

In [ ]:
def edge(img):
    #高斯模糊,降低噪声
    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    #灰度图像
    gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)
    #图像梯度
    xgrad = cv2.Sobel(gray, cv2.CV_16SC1, 1, 0)
    ygrad = cv2.Sobel(gray, cv2.CV_16SC1, 0, 1)
    #计算边缘，50和150参数必须符合1：3或者1：2，用粗筛判定低阈值
    edge_output = cv2.Canny(xgrad, ygrad, 35, 105)
    return edge_output

In [ ]:
EdgeSav = edge(rawImage)
cv2.imwrite(r'E:\Machine Learning\findContours\test3.jpg', EdgeSav)

In [ ]:
'''
#效果图，批量时不运行
cv2.namedWindow('edge', 0)
cv2.imshow("edge", EdgeSav)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

## 轮廓提取

In [2]:
EdgeRef = cv2.imread(r'E:\Machine Learning\cannyfile\DSCN1394.JPG')
#GaussianBlur去除部分的干扰
blurred = cv2.GaussianBlur(EdgeRef, (3, 3), 0)
gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)

In [3]:
#Sobel算子，图像边缘检测，
Sobel_x = cv2.Sobel(gray, cv2.CV_16S, 1, 0)  #（x方向）
Sobel_y = cv2.Sobel(gray, cv2.CV_16S, 0, 1)  #（y方向）

gradX = cv2.convertScaleAbs(Sobel_x)  # 转回uint8
gradY = cv2.convertScaleAbs(Sobel_y)

gradient = cv2.addWeighted(gradX, 0.5, gradY, 0.5, 0)
#gradient = gradX
#gradient = gradY

In [7]:
ret, binary = cv2.threshold(gradient, 5, 255, cv2.THRESH_BINARY)
kernelX = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
image = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernelX)

In [ ]:
#二值化开操作
#ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV) #阈值
kernelX = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
image = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernelX)

In [9]:
#膨胀腐蚀，通过膨胀连接相近的图像区域，通过腐蚀去除孤立细小的色块。（闭操作again，细调）
kernelX = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
kernelY = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 19))


image = cv2.dilate(image, kernelX)
image = cv2.erode(image, kernelX)

image = cv2.dilate(image, kernelY)
image = cv2.erode(image, kernelY)

In [16]:
#中值滤波，滤去噪声
image = cv2.medianBlur(image, 5)

In [14]:
rawImage = r'E:\Machine Learning\findContours\DSCN1394.JPG'

In [15]:
contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))
image = cv2.drawContours(rawImage, [box], -1, (0, 0, 255), 3)

TypeError: Expected cv::UMat for argument 'image'

## 原图剪切

# 效果显示

In [13]:
cv2.namedWindow('image', 0)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()